In [ ]:
import requests
import time
from google.colab import files
from IPython.display import Markdown
import getpass  # Secure API input
md_text = """
### 🔑 Create your AssemblyAI API key (if not already created)
➡️ [**Get your API Key here**](https://www.assemblyai.com)
"""
display(Markdown(md_text))
# Secure input for API Key
api_key = getpass.getpass("Enter your AssemblyAI API key: ")

# Upload the audio file
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

def transcribe_audio(file_name, api_key):
    headers = {'authorization': api_key}
    # Upload
    with open(file_name, 'rb') as f:
        response = requests.post("https://api.assemblyai.com/v2/upload", headers=headers, files={"file": f})
    if response.status_code != 200:
        return "Error: File upload failed."
    upload_url = response.json()["upload_url"]
# Request transcription
    response = requests.post("https://api.assemblyai.com/v2/transcript", headers=headers, json={"audio_url": upload_url})
    if response.status_code != 200:
        return "Error: Transcription request failed."
    transcript_id = response.json()["id"]
    print("🔄 Transcription in progress...") # Check status
    while True:
        response = requests.get(f"https://api.assemblyai.com/v2/transcript/{transcript_id}", headers=headers)
        result = response.json()
        if result["status"] == "completed":
            return result["text"]
        elif result["status"] == "failed":
            return "Error: Transcription failed."
        time.sleep(5)

print("\n🔊 Transcribing...")# Transcribe
transcription = transcribe_audio(file_name, api_key)
print("\n✅ **Transcription Result:**\n")
print(transcription)
# Save Transcripted text to File
with open("transcription.txt", "w") as f:
    f.write(transcription)
print("\n💾 Transcription saved as 'transcription.txt'!")
